In [1]:
import numpy as np
import pandas as pd
import os

# Совместим всю фотометрику в одну таблицу и добавим колонку с оптическим цветом

In [2]:
photo_data = os.listdir("photo_sdss_data")
df = pd.DataFrame()
for entry in photo_data:
    fname = "photo_sdss_data/" + entry
    gr = pd.read_csv(fname)
    gr['iGrID'] = fname[:-4].split('_')[-1]
    df = pd.concat([gr, df]).reset_index(drop=True)
df['optic_color'] = np.abs(df['dered_g']-df['dered_r'])

# Теперь загрузим все данные из SDSS по группам с $ Ntot > 30$ (они были отфильтрованы так ещё при сохранении)

In [3]:
glist_s = pd.read_csv('./glist_s.csv')
glist_s.rename(columns={'Name':'objid'}, inplace=True)
glist_s['objid'] = glist_s['objid'].apply(lambda x: int(x[2:-1]))


# Оставим данные по галактикам, которые входят в оба датафрейма и имеют ненулевую dereg_g - dereg_r

In [4]:
glist_merged = pd.merge(glist_s, df, on='objid', how='left',suffixes=('_s','_photo'))
glist_merged = glist_merged[pd.notnull(glist_merged['optic_color'])]

In [5]:
z_photo = glist_merged[['objid', 'ra', 'dec','z_gal','optic_color','iGrID_photo']]
z_photo.rename(columns={'iGrID_photo':'iGrID'}, inplace=True)

/home/igor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [6]:
z_photo.head(10)

,objid,ra,dec,z_gal,optic_color,iGrID
1,1237651225709576232,259.240503,57.480007,0.028285,0.69299,51
3,1237651225172771115,258.607534,57.154777,0.029271,0.65348,51
4,1237651212287738211,258.724686,57.462433,0.029612,0.43636,51
5,1237651212287738237,258.628668,57.430487,0.030457,0.11510,51
6,1237651212287738259,258.891623,57.450273,0.030010,0.77901,51
7,1237651225172770970,258.619935,57.249860,0.028236,0.51583,51
10,1237651212287803711,258.811845,57.382188,0.025598,0.74364,51
11,1237651212287803641,258.947525,57.301870,0.026489,0.78801,51
14,1237651212287803549,258.852002,57.322507,0.029370,0.96047,51
15,1237651225172640033,258.492818,57.486888,0.030262,0.32741,51


In [7]:
z_photo.to_csv("z_photo_data.csv", index=False)